In [114]:
import warnings
warnings.filterwarnings('ignore')

In [115]:
import pandas as pd
import sqlite3
import pickle

In [ ]:
conn = sqlite3.connect('cleaned.db')
df = pd.read_sql_query("SELECT id, summary, title, authors, subjects, bookshelves from llama_embeddings LEFT", conn, index_col='id')

In [135]:
# df['text'] = [pickle.loads(text) for text in df['text']]
for column in ['summary', 'title', 'authors', 'subjects',   'bookshelves']:
    df[column] = [pickle.loads(text) for text in df[column]]

In [136]:
print(f'Number of rows: {len(df)}')
df.head()

Number of rows: 15253


,summary,title,authors,subjects,bookshelves
id,,,,,
1,"[-0.10176070034503937, 0.007942539639770985, 0...","[-0.01026034913957119, 0.009973442181944847, 0...","[[-0.011376812122762203, -0.006715957075357437...","[[-0.004802127368748188, -0.014699180610477924...","[[-0.04011578485369682, 0.04477154091000557, 0..."
2,"[-0.05642512068152428, 0.016936874017119408, -...","[-0.0936107188463211, -0.05000295490026474, 0....","[[0.003930508624762297, -0.021613741293549538,...","[[-0.019040683284401894, -0.04767512157559395,...","[[-0.04011578485369682, 0.04477154091000557, 0..."
3,"[-0.04787103459239006, 0.043358348309993744, 0...","[-0.06682515889406204, -0.00882570631802082, 0...","[[-0.08240365236997604, 0.021866029128432274, ...","[[-0.07022667676210403, 0.006524343509227037, ...","[[-0.028188010677695274, 0.03697013482451439, ..."
4,"[0.020010314881801605, -0.0032934043556451797,...","[-0.046828411519527435, 0.04849471524357796, 0...","[[-0.028349613770842552, 0.06836843490600586, ...","[[-0.06208223104476929, 0.03326922655105591, 0...","[[-0.010167899541556835, -0.008022290654480457..."
5,"[-0.04907809942960739, -0.041994184255599976, ...","[-0.03496657684445381, -0.043803293257951736, ...","[[0.003930508624762297, -0.021613741293549538,...","[[-0.029587898403406143, 0.005127390846610069,...","[[0.003930508624762297, -0.021613741293549538,..."


# Recommendations

In [175]:
result['cluster'] = labels
result = result.copy()

In [212]:
from sklearn.metrics.pairwise import cosine_similarity

# result.set_index(['id'])

def compute_score(scoring_book, rated_book, rating, cosine_weight = 0.5, cluster_weight = 0.5):
    sb = clusterable.loc[scoring_book]
    rb = clusterable.loc[rated_book]
    score = rating * (cosine_weight * cosine_similarity(sb.array.reshape(1, -1), rb.array.reshape(1, -1))[0][0])
    return score

In [199]:
clusterable.loc[20].array

<NumpyExtensionArray>
[                  11.0,   0.005110925529152155,  0.0010671572526916862,
  -0.019956283271312714,   0.023811297491192818,  0.0012056599371135235,
   0.049638472497463226, -0.0009032613015733659,     0.0248134583234787,
   0.005342364776879549,
 ...
    0.03446990344673395,    0.06039569666609168,   -0.15237700007855892,
 -0.0036830524913966656,  -0.006131426664069295,   -0.03912099730223417,
    -0.1352720521390438,   -0.10496517503634095,    0.03994312975555658,
 -0.0023058028891682625]
Length: 1921, dtype: float64

In [213]:
compute_score(24, 20, 1)

0.4381849503177405

In [243]:
books = [
    (1, 15),
    (-1, 19),
    (-1, 17),
    (1, 42)
]
ratings = pd.DataFrame({
    'rating': [0] * len(result)
}, index=result.index)
for score, book_id in books:
    book = result.loc[book_id]
    potentials = result[result['cluster'] == book['cluster']]
    for index, potential in potentials.iterrows():
        r = potential
        ratings.loc[index, 'rating'] += compute_score(book_id, index, score)

ratings

,rating
id,
1,-0.471522
2,-0.470914
3,0.000000
4,0.000000
5,-0.470856
...,...
16058,0.000000
16059,0.000000
16060,0.000000


In [245]:
sorted_ratings = ratings.sort_values('rating',ascending = False)
sorted_ratings[0:5]

,rating
id,
42,0.500000
15,0.500000
2701,0.498306
2489,0.498093
43,0.497480


In [253]:
def get_book(id):
    return result[id]

In [254]:
pd.concat([get_book(r) for r in sorted_ratings[0:5].index])

KeyError: 42

In [255]:
get_book(1)

KeyError: 1

In [ ]:
clusters = pd.read_sql_query("SELECT id, centroid from clusters", conn, index_col='id')

In [ ]:
def compute_score(embedding1, embedding2):
    return cosine_similarity(embedding1.array.reshape(1, -1), embedding2.array.reshape(1, -1))[0][0]

In [ ]:
def get_candidates(books):
    scores = []
    for cluster in clusters:
        cluster_center = clusters.loc[cluster]
        score = 0
        for book in books:
            score += book[0] * compute_score(book[1], cluster_center)
        scores.append((cluster, score))
    return sorted(scores, key=lambda x: x[1], reverse=True)[0:10]

In [ ]:
def get_recommendations(books):
    candidates = get_candidates(books)
    recommendations = []
    candidates = [df[df['cluster'] == candidate[0]] for candidate in candidates]
    for candidate in candidates:
        score = 0
        for book in books:
            score += book[0] * compute_score(book[1], candidate)
        recommendations.append((candidate, score))
    return sorted(recommendations, key=lambda x: x[1], reverse=True)[0:5]

In [ ]:
get_recommendations([
    (1, 1),
    (-1, 2),
    (1, 3)
])